In [1]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 
import numpy as np
import matplotlib.pyplot as plt
!pip install snips
import snips as snp

In [2]:
train=pd.read_csv("C:/Users/Tarun/Desktop/Kaggle/titanic/train.csv")

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
X=train

for dataset in X:
    X['Title'] = X.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    
for dataset in X:
    X['Title'] = X['Title'].replace(['Lady','Countess','Capt','Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'],'Rare')

    X['Title'] = X['Title'].replace('Mlle', 'Miss')
    X['Title'] = X['Title'].replace('Ms', 'Miss')
    X['Title'] = X['Title'].replace('Mme', 'Mrs')

Y=X.Survived
X.drop(["Survived","PassengerId", "Name","Ticket","Cabin"], axis = 1, inplace = True) 


In [5]:
X.tail()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
886,2,male,27.0,0,0,13.00,S,Rare
887,1,female,19.0,0,0,30.00,S,Miss
888,3,female,NaN,1,2,23.45,S,Miss
889,1,male,26.0,0,0,30.00,C,Mr
890,3,male,32.0,0,0,7.75,Q,Mr


In [6]:
#clean+norm

X.Title=X.Title.fillna("Mr")

X.Pclass=X.Pclass.fillna(2)

X.Sex=X.Sex.fillna("male")
mapd={'male':0,'female':1}
X.Sex=X.Sex.map(mapd)

X.Age=X.Age.fillna(X.Age.mean())
a=max(X.Age)
X.Age/=a

X.SibSp=X.SibSp.fillna(X.SibSp.mean())
si=max(X.SibSp)
X.SibSp/=si

X.Parch=X.Parch.fillna(X.Parch.mean())
pa=max(X.Parch)
X.Parch/=pa

X.Fare=X.Fare.fillna(X.Fare.mean())
d=max(X.Fare)
X.Fare/=d

X.Embarked=X.Embarked.fillna("S")

X['a']=X.Pclass*X.Sex
pop1=max(X['a'])
X['a']/=pop1
X['b']=X.Pclass*X.Age
pop2=max(X['b'])
X['b']/=pop2
X['c']=X.Sex*X.Age
pop3=max(X['c'])
X['c']/=pop3

columnTransformer = ColumnTransformer([('encoder',OneHotEncoder(),[0,1,6,7])],remainder='passthrough') 
X = np.array(columnTransformer.fit_transform(X), dtype = np.float) 

#X = np.delete(X, [15, 21, 11, 9, 12], axis=1)

In [7]:
mlp = MLPClassifier(hidden_layer_sizes=(320,160,80,40,20,10,5),activation='relu',solver='lbfgs',max_iter=5000,alpha=0.95,learning_rate='adaptive',random_state=0)

In [8]:
mlp.fit(X,Y)

MLPClassifier(activation='relu', alpha=0.95, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(320, 160, 80, 40, 20, 10, 5),
       learning_rate='adaptive', learning_rate_init=0.001, max_iter=5000,
       momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
       power_t=0.5, random_state=0, shuffle=True, solver='lbfgs',
       tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [9]:
test=pd.read_csv("C:/Users/Tarun/Desktop/Kaggle/titanic/test.csv", index_col=None)

In [10]:
XT=test
a=test.PassengerId

for dataset in XT:
    XT['Title'] = XT.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    
for dataset in XT:
    XT['Title'] = XT['Title'].replace(['Lady','Countess','Capt','Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'],'Rare')

    XT['Title'] = XT['Title'].replace('Mlle', 'Miss')
    XT['Title'] = XT['Title'].replace('Ms', 'Miss')
    XT['Title'] = XT['Title'].replace('Mme', 'Mrs')


XT.drop(["PassengerId", "Name","Ticket","Cabin"], axis = 1, inplace = True)



XT.Title=XT.Title.fillna("Mr")

XT.Pclass=XT.Pclass.fillna(2)

XT.Sex=XT.Sex.fillna("male")
XT.Sex=XT.Sex.map(mapd)

XT.Age=XT.Age.fillna(XT.Age.mean())
XT.Age/=a

XT.SibSp=XT.SibSp.fillna(XT.SibSp.mean())
XT.SibSp/=si

XT.Parch=XT.Parch.fillna(XT.Parch.mean())
XT.Parch/=pa


XT.Fare=XT.Fare.fillna(XT.Fare.mean())
XT.Fare/=d

XT.Embarked=XT.Embarked.fillna("S") 


XT['a']=XT.Pclass*XT.Sex
XT.a/=pop1
XT['b']=XT.Pclass*XT.Age
XT.b/=pop2
XT['c']=XT.Sex*XT.Age
XT.c/=pop3

columnTransformer = ColumnTransformer([('encoder',OneHotEncoder(),[0,1,6,7])],remainder='passthrough') 
XT = np.array(columnTransformer.fit_transform(XT), dtype = np.float) 

#XT = np.delete(XT, [15, 21, 11, 9, 12], axis=1)

In [11]:
pred=mlp.predict(XT)
predi={'PassengerId':a,'Survived':pred}
final=pd.DataFrame(predi)
final.to_csv('C:/Users/Tarun/Desktop/Kaggle/titanic/predi.csv', index=False)

In [12]:
mlp.score(X,Y)

0.9326599326599326

In [13]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression as lr

In [14]:
logreg=lr(solver='lbfgs')
logreg.fit(X,Y)
rfe = RFE(logreg,10)
fit = rfe.fit(X,Y)
print("Num Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % fit.ranking_)
logreg.score(X,Y)

Num Features: 10
Selected Features: [ True False False  True  True False False False  True False  True False
  True  True  True  True False  True False False]
Feature Ranking: [ 1 11  2  1  1  9 10  5  1  7  1  4  1  1  1  1  6  1  3  8]


0.8383838383838383

In [15]:
X=pd.DataFrame(X)
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.2750,0.125,0.0,0.014151,0.000000,0.297297,0.000000
1,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.4750,0.125,0.0,0.139136,0.333333,0.171171,0.603175
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.3250,0.000,0.0,0.015469,1.000000,0.351351,0.412698
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.4375,0.125,0.0,0.103644,0.333333,0.157658,0.555556
4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.4375,0.000,0.0,0.015713,0.000000,0.472973,0.000000


In [16]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
k_best = SelectKBest(f_classif, k=5)
k_best.fit_transform(X, Y)
p_values = pd.DataFrame({'column': X.columns, 'p_value':k_best.pvalues_}).sort_values('p_value')

In [17]:
p_values

,column,p_value
10,10,2.428783e-71
3,3,1.406066e-69
4,4,1.406066e-69
19,19,2.373133e-57
17,17,4.207642e-31
11,11,7.552395e-26
9,9,6.694006e-25
18,18,1.476837e-23
2,2,5.510281e-23
0,0,3.190582e-18


In [18]:
plt.plot(mlp.loss_curve_)

AttributeError: 'MLPClassifier' object has no attribute 'loss_curve_'

In [ ]:
"""from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X,Y)

"""

In [ ]:
#rf_random.best_params_

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=160,min_samples_split=5,min_samples_leaf=2,max_features='sqrt',max_depth=10,bootstrap=True,random_state=0)
clf.fit(X,Y)
clf.score(X,Y)


In [ ]:
#pred=clf.predict(XT)
#predi={'PassengerId':a,'Survived':pred}
#final=pd.DataFrame(predi)
#final.to_csv('C:/Users/Tarun/Desktop/Kaggle/titanic/predi.csv', index=False)

In [ ]:
#rf_random.score(X,Y)